In [1]:
import torch 
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Prepare SR25 Dataset

In [2]:
import copy
import numpy as np
import torch.nn as nn
import networkx as nx
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.transforms import VirtualNode
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool, PNAConv
from torch_geometric.utils import from_networkx, to_networkx, to_undirected, to_scipy_sparse_matrix, degree
from torch_geometric.data.data import Data
from sklearn.model_selection import train_test_split

In [4]:
import pickle
import os

from torch_geometric.data import InMemoryDataset

class SRDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(SRDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["sr251256.g6"]  #sr251256  sr351668

    @property
    def processed_file_names(self):
        return 'data.pt'

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        # Read data into huge `Data` list. 
        b=self.processed_paths[0]   
        dataset = nx.read_graph6(self.raw_paths[0])
        data_list = []
        for i,datum in enumerate(dataset):
            x = torch.ones(datum.number_of_nodes(),1)
            edge_index = to_undirected(torch.tensor(list(datum.edges())).transpose(1,0))            
            data_list.append(Data(edge_index=edge_index, x=x, y=0))
            
        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [33]:
class SpectralDesign(object):   

    def __init__(self,nmax=0,recfield=1,dv=5,nfreq=5,adddegree=False,laplacien=True,addadj=False,vmax=None):
        # receptive field. 0: adj, 1; adj+I, n: n-hop area 
        self.recfield=recfield  
        # b parameter
        self.dv=dv
        # number of sampled point of spectrum
        self.nfreq=  nfreq
        # if degree is added to node feature
        self.adddegree=adddegree
        # use laplacian or adjacency for spectrum
        self.laplacien=laplacien
        # add adjacecny as edge feature
        self.addadj=addadj
        # use given max eigenvalue
        self.vmax=vmax

        # max node for PPGN algorithm, set 0 if you do not use PPGN
        self.nmax=nmax    

    def __call__(self, data):

        n =data.x.shape[0]     
        nf=data.x.shape[1]  


        data.x=data.x.type(torch.float32)  
               
        nsup=self.nfreq+1
        if self.addadj:
            nsup+=1
            
        A=np.zeros((n,n),dtype=np.float32)
        SP=np.zeros((nsup,n,n),dtype=np.float32) 
        A[data.edge_index[0],data.edge_index[1]]=1
        
        if self.adddegree:
            data.x=torch.cat([data.x,torch.tensor(A.sum(0)).unsqueeze(-1)],1)

        # calculate receptive field. 0: adj, 1; adj+I, n: n-hop area
        if self.recfield==0:
            M=A
        else:
            M=(A+np.eye(n))
            for i in range(1,self.recfield):
                M=M.dot(M) 

        M=(M>0)

        
        d = A.sum(axis=0) 
        # normalized Laplacian matrix.
        dis=1/np.sqrt(d)
        dis[np.isinf(dis)]=0
        dis[np.isnan(dis)]=0
        D=np.diag(dis)
        nL=np.eye(D.shape[0])-(A.dot(D)).T.dot(D)
        V,U = np.linalg.eigh(nL) 
        V[V<0]=0
        # keep maximum eigenvalue for Chebnet if it is needed
        data.lmax=V.max().astype(np.float32)

        if not self.laplacien:        
            V,U = np.linalg.eigh(A)

        # design convolution supports
        vmax=self.vmax
        if vmax is None:
            vmax=V.max()

        freqcenter=np.linspace(V.min(),vmax,self.nfreq)
        
        # design convolution supports (aka edge features)         
        for i in range(0,len(freqcenter)): 
            SP[i,:,:]=M* (U.dot(np.diag(np.exp(-(self.dv*(V-freqcenter[i])**2))).dot(U.T))) 
        # add identity
        SP[len(freqcenter),:,:]=np.eye(n)
        # add adjacency if it is desired
        if self.addadj:
            SP[len(freqcenter)+1,:,:]=A
           
        # set convolution support weigths as an edge feature
        E=np.where(M>0)
        data.edge_index2=torch.Tensor(np.vstack((E[0],E[1]))).type(torch.int64)
        data.edge_attr2 = torch.Tensor(SP[:,E[0],E[1]].T).type(torch.float32)  

        # set tensor for Maron's PPGN         
        if self.nmax>0:       
            H=torch.zeros(1,nf+2,self.nmax,self.nmax)
            H[0,0,data.edge_index[0],data.edge_index[1]]=1 
            H[0,1,0:n,0:n]=torch.diag(torch.ones(data.x.shape[0]))
            for j in range(0,nf):      
                H[0,j+2,0:n,0:n]=torch.diag(data.x[:,j])
            data.X2= H 
            M=torch.zeros(1,2,self.nmax,self.nmax)
            for i in range(0,n):
                M[0,0,i,i]=1
            M[0,1,0:n,0:n]=1-M[0,0,0:n,0:n]
            data.M= M        

        return data

In [43]:
transform = SpectralDesign(nmax=25,recfield=1,dv=2,nfreq=5,adddegree=True)
SR25_dataset = SRDataset(root="../dataset/sr25/", pre_transform=transform)

SR25_dataset.data.y = torch.arange(len(dataset.data.y)).long() 

/tmp/ipykernel_2176769/3868519769.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths[0])


# Add Isomorphic Pairs

In [44]:
def add_isomorphic_pairs_dgl(dataset, num_pairs=5):
    isomorphic_pair = []
    original_indices = []

    for i in range(num_pairs):
        # Pick a random graph from the dataset
        original_graph = random.choice(dataset)
        original_idx = dataset.index(original_graph)

        # Convert to NetworkX and create isomorphic graphs
        G = dgl.to_networkx(original_graph)
        nodes = list(G.nodes())
        random.shuffle(nodes)  # Shuffle to get isomorphic graph
        mapping = {node: nodes[i] for i, node in enumerate(nodes)}
        isomorphic_G = nx.relabel_nodes(G, mapping)

        # Convert back to DGL
        isomorphic_dgl_graph = dgl.from_networkx(isomorphic_G)
        isomorphic_dgl_graph.ndata['x'] = original_graph.ndata['x']  # Copy node features

        isomorphic_pair.append(isomorphic_dgl_graph)
        original_indices.append(original_idx)

    return dataset, isomorphic_pair, original_indices

# Graph Transformations

In [46]:
# VN
transform = VirtualNode()
def apply_vn(dgl_graphs):
  vn_EXP_dgl = []
  for graph in dgl_graphs:
    graph_pyg = from_dgl(graph)
    graph_pyg_copy = copy.deepcopy(graph_pyg)
    graph_vn = transform(graph_pyg_copy)
    graph_vn_dgl = to_dgl(graph_vn)
    vn_EXP_dgl.append(graph_vn_dgl)

  return vn_EXP_dgl

# Centrality
def add_centrality_to_node_features(dgl_graph, centrality_measure='degree'):
    # Convert DGL data to NetworkX graph
    G = dgl_graph.to_networkx()
    G = nx.Graph(G)

    # Compute the centrality measure
    if centrality_measure == 'degree':
        centrality = nx.degree_centrality(G)
    elif centrality_measure == 'closeness':
        centrality = nx.closeness_centrality(G)
    elif centrality_measure == 'betweenness':
        centrality = nx.betweenness_centrality(G)
    elif centrality_measure == 'eigenvector':
        if not nx.is_connected(G):
        # Handle connected components separately
            centrality = {}
            for component in nx.connected_components(G):
                subgraph = G.subgraph(component)
                sub_centrality = nx.eigenvector_centrality(subgraph, max_iter=500, tol=1e-4)
                centrality.update(sub_centrality)
        else:
            centrality = nx.eigenvector_centrality(G, max_iter=500, tol=1e-4)
    else:
        raise ValueError(f'Unknown centrality measure: {centrality_measure}')

    # Convert centrality to tensor and add as node feature
    centrality_values = np.array([centrality[node] for node in range(dgl_graph.number_of_nodes())], dtype=np.float32).reshape(-1, 1)
    centrality_values = torch.round(torch.tensor(centrality_values) * 10000) / 10000
    # Concatenate the centrality with existing node features
    if 'x' in dgl_graph.ndata:
        dgl_graph.ndata['x'] = torch.cat([dgl_graph.ndata['x'], centrality_values], dim=1)
    else:
        dgl_graph.ndata['x'] = centrality_values
    return dgl_graph

# Degree
def degree_dataset(dataset):
    # Compute centrality and add it as an additional feature
    Graph_data_degree = []
    for data in dataset:
        data_copy = copy.deepcopy(data)  # Create a deep copy of the graph
        data_copy = add_centrality_to_node_features(data_copy, centrality_measure='degree')
        Graph_data_degree.append(data_copy)
    return Graph_data_degree

# Closeness
def closeness_dataset(dataset):
    # Compute centrality and add it as an additional feature
    Graph_data_clo = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        data_copy = add_centrality_to_node_features(data_copy, centrality_measure='closeness')
        Graph_data_clo.append(data_copy)
    return Graph_data_clo

#Betweenness
def betweenness_dataset(dataset):
    # Compute centrality and add it as an additional feature
    Graph_data_bet = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        data_copy = add_centrality_to_node_features(data_copy, centrality_measure='betweenness')
        Graph_data_bet.append(data_copy)
    return Graph_data_bet

# Eigenvector
def eigenvector_dataset(dataset):
    # Compute centrality and add it as an additional feature
    Graph_data_eig = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        data_copy = add_centrality_to_node_features(data_copy, centrality_measure='eigenvector')
        Graph_data_eig.append(data_copy)
    return Graph_data_eig

# DE
def add_distance_encoding(dgl_graph):
    # Compute the shortest distance matrix using dgl.shortest_dist
    dist = dgl.shortest_dist(dgl_graph).float()  # Convert to float to handle inf

    # Replace -1 with inf (to handle unreachable nodes similar to NetworkX's behavior)
    dist[dist == -1] = float('inf')

    # Calculate the average shortest distance for each node
    finite_distances = torch.where(dist == float('inf'), torch.tensor(float('nan')), dist)
    average_distance = torch.nanmean(finite_distances, dim=1).view(-1, 1)  # Use nanmean to ignore infinities

    # Add the average distance to the existing node features in the DGL graph
    if 'x' in dgl_graph.ndata:
        dgl_graph.ndata['x'] = torch.cat([dgl_graph.ndata['x'], average_distance], dim=1)
    else:
        dgl_graph.ndata['x'] = average_distance

    return dgl_graph

def distance_encoding(dataset):
    Graph_data_DE = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        data_copy = add_distance_encoding(data_copy)
        Graph_data_DE.append(data_copy)
    return Graph_data_DE

# GE
from torch_geometric.transforms import AddLaplacianEigenvectorPE

def canonicalize_eigenvectors(eigenvectors):
    """Canonicalize eigenvectors by fixing their signs for consistency."""
    for i in range(eigenvectors.shape[1]):
        if eigenvectors[0, i] < 0:  # Flip sign if the first element is negative
            eigenvectors[:, i] = -eigenvectors[:, i]
    return eigenvectors

def add_canonicalized_laplacian_pe(dgl_graph, k=5):
    """
    Add canonicalized Laplacian positional encoding to a DGL graph.

    Args:
        dgl_graph: Input DGL graph.
        k: Number of Laplacian eigenvectors to compute.

    Returns:
        dgl_graph: DGL graph with Laplacian PE appended to node features.
    """
    # Step 1: Convert DGL graph to adjacency matrix
    G = dgl_graph.to_networkx()
    G = nx.Graph(G)
    adj = nx.to_numpy_array(G)

    # Step 2: Compute Laplacian matrix
    degree_matrix = np.diag(np.sum(adj, axis=1))
    laplacian = degree_matrix - adj

    # Step 3: Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian)

    # Step 4: Select the smallest k eigenvectors (sorted by eigenvalues)
    idx = np.argsort(eigenvalues)[:k]
    eigenvectors = eigenvectors[:, idx]

    # Step 5: Canonicalize eigenvectors
    eigenvectors = canonicalize_eigenvectors(torch.tensor(eigenvectors, dtype=torch.float))

    # Step 6: Add the eigenvectors as new node features
    if 'x' in dgl_graph.ndata:
        dgl_graph.ndata['x'] = torch.cat([dgl_graph.ndata['x'], eigenvectors], dim=1)
    else:
        dgl_graph.ndata['x'] = eigenvectors

    return dgl_graph

def Graph_encoding(dataset, k=3):
    """
    Apply canonicalized Laplacian positional encoding to a list of DGL graphs.

    Args:
        dgl_graphs: List of DGL graphs.
        k: Number of Laplacian eigenvectors to compute.

    Returns:
        GE_EXP_dgl: List of DGL graphs with Laplacian PE added.
    """
    GE_EXP_dgl = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        graph_pe = add_canonicalized_laplacian_pe(data_copy, k=k)
        GE_EXP_dgl.append(graph_pe)
    return GE_EXP_dgl

# Sub
def extract_local_subgraph_features(dgl_graph, radius=2):
    # Convert PyG data to NetworkX graph
    G = dgl_graph.to_networkx()
    G = nx.Graph(G)

    # Initialize a list to store subgraph features for each node
    subgraph_sizes = []
    subgraph_degrees = []

    for node in G.nodes():
        # Extract the ego graph (subgraph) around the node
        subgraph = nx.ego_graph(G, node, radius=radius)

        # Example feature 1: Size of the subgraph (number of nodes)
        subgraph_size = subgraph.number_of_nodes()
        subgraph_sizes.append(subgraph_size)

        # Example feature 2: Average degree of the subgraph
        subgraph_degree = np.mean([d for n, d in subgraph.degree()])
        subgraph_degrees.append(subgraph_degree)

    # Convert the features to tensors and add them as node features
    subgraph_sizes_tensor = torch.tensor(subgraph_sizes, dtype=torch.float).view(-1, 1)
    subgraph_degrees_tensor = torch.tensor(subgraph_degrees, dtype=torch.float).view(-1, 1)

    if 'x' in dgl_graph.ndata:
        dgl_graph.ndata['x'] = torch.cat([dgl_graph.ndata['x'], subgraph_sizes_tensor, subgraph_degrees_tensor], dim=1)
    else:
        dgl_graph.ndata['x'] = torch.cat([subgraph_sizes_tensor, subgraph_degrees_tensor], dim=1)

    return dgl_graph

def subgraph_dataset(dataset, radius=3):
    # Compute centrality and add it as an additional feature
    Graph_data_sub = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        data_copy = extract_local_subgraph_features(data_copy, radius=radius)
        Graph_data_sub.append(data_copy)
    return Graph_data_sub

# ExN
def add_extra_node_on_each_edge(dgl_graph):
    # Collect new edges (source, destination) and the new node features
    new_edges_src = []
    new_edges_dst = []
    new_node_features = []

    # Original number of nodes
    num_original_nodes = dgl_graph.num_nodes()

    # Use a set to track edges we have already processed (to avoid duplicates)
    processed_edges = set()

    # Iterate over all edges
    for i in range(dgl_graph.num_edges()):
        u, v = dgl_graph.edges()[0][i].item(), dgl_graph.edges()[1][i].item()

        # Avoid processing reverse edges (v, u) if (u, v) is already processed
        if (u, v) in processed_edges or (v, u) in processed_edges:
            continue

        # Mark the edge as processed
        processed_edges.add((u, v))
        processed_edges.add((v, u))  # In case there is a reverse edge

        # Add a new node
        new_node_id = num_original_nodes + len(new_node_features)
        mean_feature = (dgl_graph.ndata['x'][u] + dgl_graph.ndata['x'][v]) / 2
        new_node_features.append(mean_feature)

        # Add new edges connecting the new node to the original nodes
        new_edges_src.append(u)
        new_edges_dst.append(new_node_id)

        new_edges_src.append(new_node_id)
        new_edges_dst.append(v)

    # Add new nodes to the DGL graph
    dgl_graph.add_nodes(len(new_node_features), {'x': torch.stack(new_node_features)})

    # Remove the original edges
    dgl_graph.remove_edges(torch.arange(dgl_graph.num_edges()))

    # Add new edges to the DGL graph
    dgl_graph.add_edges(new_edges_src, new_edges_dst)

    return dgl_graph

def extra_node_dataset(dataset):
    Graph_data_exN = []
    for data in dataset:
        data_copy = copy.deepcopy(data)
        dgl_graph = add_extra_node_on_each_edge(data_copy)
        Graph_data_exN.append(dgl_graph)
    return Graph_data_exN

def count_3_star(G):
    """Count 3-star graphlets for each node."""
    # A 3-star is a node with at least three neighbors
    star_counts = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degree = len(neighbors)
        # Count the number of 3-combinations of neighbors
        star_counts[node] = max(0, (degree * (degree - 1) * (degree - 2)) // 6)
    return star_counts

def count_tailed_triangle(G):
    """Count tailed triangle graphlets for each node."""
    tail_counts = {node: 0 for node in G.nodes()}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        for neighbor in neighbors:
            # For each pair of neighbors, check if there's a triangle
            for other in neighbors:
                if neighbor != other and G.has_edge(neighbor, other):
                    # Found a triangle, check for a tail
                    for extra in G.neighbors(node):
                        if extra not in {neighbor, other}:
                            tail_counts[node] += 1
    return tail_counts

def count_4_cycle(G):
    """Count 4-cycle graphlets for each node."""
    cycle_counts = {node: 0 for node in G.nodes()}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        for i, neighbor1 in enumerate(neighbors):
            for neighbor2 in neighbors[i + 1:]:
                # Check for shared neighbors forming a 4-cycle
                for shared_neighbor in G.neighbors(neighbor1):
                    if shared_neighbor in G.neighbors(neighbor2):
                        cycle_counts[node] += 1
    return cycle_counts

def graphlet_based_encoding(dgl_graph):
    """
    Add graphlet-based features (3-star, triangle, tailed triangle, 4-cycle) to node features.

    Args:
        dgl_graph: Input DGL graph.

    Returns:
        dgl_graph: DGL graph with graphlet-based features added.
    """
    # Convert DGL graph to NetworkX
    G = dgl_graph.to_networkx()
    G = nx.Graph(G)

    # Count graphlets
    triangle_counts = nx.triangles(G)  # Triangle counts
    star_counts = count_3_star(G)  # 3-star graphlets
    tail_counts = count_tailed_triangle(G)  # Tailed triangles
    cycle_counts = count_4_cycle(G)  # 4-cycles

    # Combine features into tensors
    num_nodes = dgl_graph.num_nodes()
    triangle_tensor = torch.tensor([triangle_counts[node] for node in range(num_nodes)], dtype=torch.float).view(-1, 1)
    star_tensor = torch.tensor([star_counts[node] for node in range(num_nodes)], dtype=torch.float).view(-1, 1)
    tail_tensor = torch.tensor([tail_counts[node] for node in range(num_nodes)], dtype=torch.float).view(-1, 1)
    cycle_tensor = torch.tensor([cycle_counts[node] for node in range(num_nodes)], dtype=torch.float).view(-1, 1)

    # Concatenate all graphlet features
    graphlet_features = torch.cat([triangle_tensor, star_tensor, tail_tensor, cycle_tensor], dim=1)

    # Add to node features
    if 'x' in dgl_graph.ndata:
        dgl_graph.ndata['x'] = torch.cat([dgl_graph.ndata['x'], graphlet_features], dim=1)
    else:
        dgl_graph.ndata['x'] = graphlet_features

    return dgl_graph

def graphlet_encoding_dataset(dgl_dataset):
    """
    Apply graphlet-based encoding to a list of DGL graphs.

    Args:
        dgl_dataset: List of DGL graphs.

    Returns:
        encoded_dataset: List of DGL graphs with graphlet-based features added.
    """
    encoded_dataset = []
    for dgl_graph in dgl_dataset:
        graph_copy = copy.deepcopy(dgl_graph)
        graph_encoded = graphlet_based_encoding(graph_copy)
        encoded_dataset.append(graph_encoded)
    return encoded_dataset

## SR25 original

In [48]:
import dgl
from torch_geometric.utils import to_dgl, from_dgl

SR25_dgl_graphs = []
for data in SR25_dataset:
    dgl_graph = to_dgl(data)
    SR25_dgl_graphs.append(dgl_graph)

# Step 4: Save the list of DGL graphs using pickling
output_file = '../data/SR25/SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_dgl_graphs, f)

print(f"Converted {len(SR25_dgl_graphs)} graphs to DGL format and saved to {output_file}.")

Converted 15 graphs to DGL format and saved to ../data/SR25/SR25_dataset.pkl.


## SR25 dummy with isomorphic pairs

In [51]:
SR25, SR25_iso, SR25_indices = add_isomorphic_pairs_dgl(SR25_dgl_graphs, num_pairs=15)

SR25_dummy_dgl = SR25 + SR25_iso

output_file = '../data/SR25/SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_dummy_dgl, f)

print(f"Converted {len(SR25_dummy_dgl)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")

Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/SR25_dataset_dummy.pkl.


## VN on SR25 original and SR25 dummy

In [52]:
vn_SR25_dgl = apply_vn(SR25_dgl_graphs)

output_file = '../data/SR25/vn_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(vn_SR25_dgl, f)

print(f"Converted {len(vn_SR25_dgl)} graphs to DGL format and saved to {output_file}.")


vn_SR25_dgl_dummy = apply_vn(SR25_dummy_dgl)
output_file = '../data/SR25/vn_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(vn_SR25_dgl_dummy, f)

print(f"Converted {len(vn_SR25_dgl_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/vn_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/vn_SR25_dataset_dummy.pkl.


## Degree Centrality on SR25 original and SR25 dummy

In [53]:
SR25_deg = degree_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/deg_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_deg, f)

print(f"Converted {len(SR25_deg)} graphs to DGL format and saved to {output_file}.")


SR25_deg_dummy = degree_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/deg_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_deg_dummy, f)

print(f"Converted {len(SR25_deg_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/deg_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/deg_SR25_dataset_dummy.pkl.


## Closeness Centrality on SR25 original and SR25 dummy

In [54]:
SR25_clo = closeness_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/clo_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_clo, f)

print(f"Converted {len(SR25_clo)} graphs to DGL format and saved to {output_file}.")


SR25_clo_dummy = closeness_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/clo_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_clo_dummy, f)

print(f"Converted {len(SR25_clo_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/clo_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/clo_SR25_dataset_dummy.pkl.


## Betweenness Centrality on SR25 original and SR25 dummy

In [55]:
SR25_bet = betweenness_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/bet_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_bet, f)

print(f"Converted {len(SR25_bet)} graphs to DGL format and saved to {output_file}.")


SR25_bet_dummy = betweenness_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/bet_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_bet_dummy, f)

print(f"Converted {len(SR25_bet_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/bet_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/bet_SR25_dataset_dummy.pkl.


## Eigenvector Centrality on SR25 original and SR25 dummy

In [56]:
SR25_eig = eigenvector_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/eig_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_eig, f)

print(f"Converted {len(SR25_eig)} graphs to DGL format and saved to {output_file}.")

SR25_eig_dummy = eigenvector_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/eig_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_eig_dummy, f)

print(f"Converted {len(SR25_eig_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/eig_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/eig_SR25_dataset_dummy.pkl.


## Distance Encoding on SR25 original and SR25 dummy

In [57]:
SR25_DE = distance_encoding(SR25_dgl_graphs)

output_file = '../data/SR25/DE_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_DE, f)

print(f"Converted {len(SR25_DE)} graphs to DGL format and saved to {output_file}.")


SR25_DE_dummy = distance_encoding(SR25_dummy_dgl)

output_file = '../data/SR25/DE_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_DE_dummy, f)

print(f"Converted {len(SR25_DE_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/DE_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/DE_SR25_dataset_dummy.pkl.


## Graph Encoding on SR25 original and SR25 dummy`

In [58]:
SR25_GE = Graph_encoding(SR25_dgl_graphs, k=3)

output_file = '../data/SR25/GE_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_GE, f)

print(f"Converted {len(SR25_GE)} graphs to DGL format and saved to {output_file}.")


SR25_GE_dummy = Graph_encoding(SR25_dummy_dgl, k=3)

output_file = '../data/SR25/GE_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_GE_dummy, f)

print(f"Converted {len(SR25_GE_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/GE_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/GE_SR25_dataset_dummy.pkl.


## Subgraph Extraction on SR25 original and SR25 dummy

In [61]:
SR25_sub = subgraph_dataset(SR25_dgl_graphs, radius=3)

output_file = '../data/SR25/sub_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_sub, f)

print(f"Converted {len(SR25_sub)} graphs to DGL format and saved to {output_file}.")


SR25_sub_dummy = subgraph_dataset(SR25_dummy_dgl, radius=3)

output_file = '../data/SR25/sub_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_sub_dummy, f)

print(f"Converted {len(SR25_sub_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/sub_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/sub_SR25_dataset_dummy.pkl.


## Extra Node on SR25 original and SR25 dummy

In [62]:
SR25_exN = extra_node_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/exN_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_exN, f)

print(f"Converted {len(SR25_exN)} graphs to DGL format and saved to {output_file}.")


SR25_exN_dummy = extra_node_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/exN_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_exN_dummy, f)

print(f"Converted {len(SR25_exN_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/exN_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/exN_SR25_dataset_dummy.pkl.


## Graphlet-Based Encoding on SR25 original and SR25 dummy

In [63]:
SR25_GLE = graphlet_encoding_dataset(SR25_dgl_graphs)

output_file = '../data/SR25/GLE_SR25_dataset.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_GLE, f)

print(f"Converted {len(SR25_GLE)} graphs to DGL format and saved to {output_file}.")

SR25_GLE_dummy = graphlet_encoding_dataset(SR25_dummy_dgl)

output_file = '../data/SR25/GLE_SR25_dataset_dummy.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(SR25_GLE_dummy, f)

print(f"Converted {len(SR25_GLE_dummy)} graphs (including isomorphisms) to DGL format and saved to {output_file}.")


Converted 15 graphs to DGL format and saved to ../data/SR25/GLE_SR25_dataset.pkl.
Converted 30 graphs (including isomorphisms) to DGL format and saved to ../data/SR25/GLE_SR25_dataset_dummy.pkl.


# Equivalence Class

In [124]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GINConv, SumPooling, PNAConv
import networkx as nx
import random
import pickle
import numpy as np
import glob

# Define a GIN model
class GIN(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers):
        super(GIN, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Input layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(in_feats, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GINConv(
                nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim)
                ), 'sum'))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Output layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(hidden_dim, out_dim),
                nn.ReLU(),
                nn.Linear(out_dim, out_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(out_dim))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 1000) / 1000
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            h = layer(g, h)
            h = batch_norm(h)
            h = F.relu(h)
            #h = torch.round(h * 1000) / 1000
            h = h/torch.sum(h) * 10
        #g_embedding = self.pool(g, h)
        return h

# Define a PNA model
class PNA(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers, aggregators, scalers, deg):
        super(PNA, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(PNAConv(in_feats, hidden_dim, aggregators, scalers, deg))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(PNAConv(hidden_dim, hidden_dim, aggregators, scalers, deg))

        # Output layer
        self.layers.append(PNAConv(hidden_dim, out_dim, aggregators, scalers, deg))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 1000) / 1000
        for layer in self.layers:
            h = layer(g, h)
            h = F.relu(h)
            #h = torch.round(h * 1000) / 1000
            h = h/torch.sum(h) * 10
        #g_embedding = self.pool(g, h)
        return h

# Define a DeepSet model
class DeepSet(nn.Module):
    def __init__(self, input_dim):
        super(DeepSet, self).__init__()
        self.input_dim = input_dim  # Store input dimension
        # First neural network to map node features to node embeddings
        self.phi = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )
        # Second neural network to map summed node embeddings to final graph embedding
        self.rho = nn.Sequential(
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # Output a single value
        )

    def forward(self, g, h):
        # Apply the first neural network to each node feature
        h = self.phi(h)
        h = torch.round(h * 10000) / 10000  # Round to reduce precision
        # Sum the node embeddings to create a graph embedding
        g_embedding = h.sum(dim=0, keepdim=True)
        # Apply the second neural network to the summed embedding
        output = self.rho(g_embedding)*100
        return output


def calculate_integer_embedding(embedding):
    # Convert embedding to a flattened list of integers
    embedding = embedding.numpy()
    embedding = np.nan_to_num(embedding, nan=0.0) * 10
    embedding = embedding.astype(int)
    if embedding.ndim == 1:
        embedding = embedding.reshape(1, -1)  # Convert 1D array to a 2D array with one row

    column_sum = embedding.sum(axis=0)
    column_mean = embedding.mean(axis=0)
    column_min = embedding.min(axis=0)

    # Concatenate the results into a single 1D array
    final_embedding = np.concatenate((column_sum, column_mean, column_min))

    return int(np.sum((final_embedding.flatten()))*10)



# Save graphs to a file
def save_graphs_to_file(graphs, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(graphs, f)

# Load graphs from a file
def load_graphs_from_file(filepath):
    with open(filepath, 'rb') as f:
        graphs = pickle.load(f)
    return graphs

# Compute equivalence classes
def compute_equivalence_classes(filepath, model, input_dim):
    graphs = load_graphs_from_file(filepath)

    # Train the model for a single epoch with a random target
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    model.train()

    for epoch_in in range(10):
        for g in graphs:
            if 'x' not in g.ndata:
                g.ndata['x'] = torch.ones(g.number_of_nodes(), input_dim) 
            optimizer.zero_grad()

            output = model(g, g.ndata['x'])  # The output is a tensor of shape (1, out_dim)
            target = torch.randn_like(output)*100  # Target is a random tensor of the same shape as output

            loss = F.mse_loss(output, target.float(), reduction='mean')  # Ensure the target is a float tensor
            loss.backward()
            optimizer.step()

    embeddings = set()

    model.eval()
    with torch.no_grad():
        for g in graphs:
            if 'x' not in g.ndata:
                g.ndata['x'] = torch.ones(g.number_of_nodes(), input_dim)  # Initialize with ones if 'h' is missing
            embedding = model(g, g.ndata['x'])
            #print("embedding: ", embedding)
            final_embedding = calculate_integer_embedding(embedding.flatten())
            embeddings.add(final_embedding)

    print("embeddings: ", embeddings)
    return len(embeddings)

# Process all .pkl files in the current directory
for filepath in glob.glob("../data/SR25/*SR25_dataset_dummy.pkl"):
    print(f"------------- Processing {filepath}...")

    # Load graphs to determine input_dim
    graphs = load_graphs_from_file(filepath)
    if 'x' in graphs[0].ndata:
        input_dim = graphs[0].ndata['x'].size(1)  # Determine the input dimension dynamically
    else:
        # If 'h' does not exist, assume a default input dimension
        input_dim = 1

    # Initialize the models
    gin_model = GIN(input_dim, hidden_dim=32, out_dim=8, num_layers=3)
    pna_model = PNA(input_dim, hidden_dim=32, out_dim=8, num_layers=3,
                    aggregators=['mean', 'max', 'sum', 'min', 'std'],
                    scalers=['identity', 'amplification', 'attenuation'],
                    deg=torch.tensor([1.0]))  # Example degree tensor

    # Compute the number of unique embeddings using GIN
    print("Computing equivalence classes using GIN model...")
    num_unique_embeddings_gin = compute_equivalence_classes(filepath, gin_model, input_dim)
    print(f"Number of unique embeddings with GIN: {num_unique_embeddings_gin}\n")

    # Compute the number of unique embeddings using PNA
    print("Computing equivalence classes using PNA model...")
    num_unique_embeddings_pna = compute_equivalence_classes(filepath, pna_model, input_dim)
    print(f"Number of unique embeddings with PNA: {num_unique_embeddings_pna}\n")

------------- Processing ../data/SR25/DE_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {2250}
Number of unique embeddings with GIN: 1

Computing equivalence classes using PNA model...
embeddings:  {2250}
Number of unique embeddings with PNA: 1

------------- Processing ../data/SR25/exN_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {90, 60}
Number of unique embeddings with GIN: 2

Computing equivalence classes using PNA model...
embeddings:  {0}
Number of unique embeddings with PNA: 1

------------- Processing ../data/SR25/sub_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {1500}
Number of unique embeddings with GIN: 1

Computing equivalence classes using PNA model...
embeddings:  {750}
Number of unique embeddings with PNA: 1

------------- Processing ../data/SR25/deg_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {1500}
Nu

# Organize Graphs in Pairs

In [68]:
from itertools import combinations

def organize_pairs_sr25(dummy_dataset, original_size, original_indices):
    """
    Organize non-isomorphic and isomorphic pairs for SR25.

    Parameters:
    - dummy_dataset: Dataset containing both original and isomorphic graphs.
    - original_size: Number of original SR25 graphs.
    - original_indices: Indices of original graphs that generated isomorphic pairs.

    Returns:
    - non_isomorphic_pairs: List of all unique non-isomorphic pairs.
    - isomorphic_pairs: List of all isomorphic pairs.
    """
    non_isomorphic_pairs = []
    isomorphic_pairs = []

    # Generate all unique non-isomorphic pairs from the original graphs
    for i, j in combinations(range(original_size), 2):
        non_isomorphic_pairs.append((dummy_dataset[i], dummy_dataset[j]))

    # Generate isomorphic pairs using the indices
    for i, original_idx in enumerate(original_indices):
        isomorphic_pairs.append((dummy_dataset[original_idx], dummy_dataset[original_size + i]))

    return non_isomorphic_pairs, isomorphic_pairs

# Distinguishing Test

In [92]:
def contrastive_loss2(embedding1, embedding2, label, margin=1.0):
    """Optimized contrastive loss: Pull embeddings together if label == 1, push them apart if label == 0."""
    euclidean_distance = F.pairwise_distance(embedding1, embedding2)
    euclidean_distance_squared = torch.pow(euclidean_distance, 2)
    
    # Ensure that label is a tensor, convert to float tensor
    if isinstance(label, int):  # Check if label is a scalar integer
        label = torch.tensor(label).float().to(embedding1.device)
    else:
        label = label.float()

    # Compute positive and negative losses
    loss_positive = euclidean_distance_squared  # For label == 1
    loss_negative = torch.pow(F.relu(margin - euclidean_distance), 2)  # For label == 0
    
    # Use torch.where with tensor inputs
    loss = torch.where(label == 1, loss_positive, loss_negative)
    return loss.mean()


def evaluate_model_with_pairs(non_isomorphic_pairs, isomorphic_pairs, model, input_dim):
    model.train()  # Set the model to training mode
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    import dgl
    from torch.utils.data import DataLoader
    
    # Custom collate function to handle DGLGraphs
    def collate_fn(batch):
        # Unpack the batch of pairs
        g1_batch, g2_batch = zip(*batch)
    
        # Batch the graphs using DGL's batch function
        batched_g1 = dgl.batch(g1_batch)
        batched_g2 = dgl.batch(g2_batch)
        
        return batched_g1, batched_g2
    
    # Custom Dataset for graph pairs (as you already have)
    class GraphPairDataset(torch.utils.data.Dataset):
        def __init__(self, pairs):
            self.pairs = pairs
    
        def __len__(self):
            return len(self.pairs)
    
        def __getitem__(self, idx):
            return self.pairs[idx]
    
    # Assuming you already have non_isomorphic_pairs and isomorphic_pairs
    non_isomorphic_dataset = GraphPairDataset(non_isomorphic_pairs)
    isomorphic_dataset = GraphPairDataset(isomorphic_pairs)
    
    # Define DataLoader with custom collate_fn
    non_isomorphic_loader = DataLoader(non_isomorphic_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
    isomorphic_loader = DataLoader(isomorphic_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
    
    # Main training loop
    for epoch in range(100):
        total_loss = 0
    
        # Iterate over non-isomorphic pairs in batches
        for batched_g1, batched_g2 in non_isomorphic_loader:
            optimizer.zero_grad()
    
            batched_g1 = batched_g1.to(device)
            batched_g2 = batched_g2.to(device)
    
            embedding1 = model(batched_g1, batched_g1.ndata['x'])
            embedding2 = model(batched_g2, batched_g2.ndata['x'])
    
            # Label is 0 for non-isomorphic pairs
            loss = contrastive_loss2(embedding1, embedding2, label=torch.zeros(batched_g1.batch_size).to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
        # Iterate over isomorphic pairs in batches
        for batched_g1, batched_g2 in isomorphic_loader:
            optimizer.zero_grad()
    
            batched_g1 = batched_g1.to(device)
            batched_g2 = batched_g2.to(device)
    
            embedding1 = model(batched_g1, batched_g1.ndata['x'])
            embedding2 = model(batched_g2, batched_g2.ndata['x'])
    
            # Label is 1 for isomorphic pairs
            loss = contrastive_loss2(embedding1, embedding2, label=torch.ones(batched_g1.batch_size).to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
        print(f"Epoch {epoch+1}/{100}, Loss: {total_loss:.4f}")

    model.eval()  # Set the model to evaluation mode
    non_isomorphic_different_count = 0
    isomorphic_same_count = 0
    isomorphic_different_count = 0

    with torch.no_grad():  # Disable gradient calculation during evaluation
        # Compare embeddings of non-isomorphic pairs
        for g1, g2 in non_isomorphic_pairs:
            g1 = g1.to(device)
            g2 = g2.to(device)
            embedding1 = model(g1, g1.ndata['x']).to(device)
            embedding2 = model(g2, g2.ndata['x']).to(device)
            cosine_sim = F.cosine_similarity(embedding1, embedding2).item()
            if cosine_sim < 0.99:
                non_isomorphic_different_count += 1  # Correctly classified as different
            #final_embedding1 = calculate_integer_embedding(embedding1)
            #final_embedding2 = calculate_integer_embedding(embedding2)
            #if final_embedding1 != final_embedding2:
            #    non_isomorphic_different_count += 1
                

        # Compare embeddings of isomorphic pairs
        for g1, g2 in isomorphic_pairs:
            g1 = g1.to(device)
            g2 = g2.to(device)
            embedding1 = model(g1, g1.ndata['x']).to(device)
            embedding2 = model(g2, g2.ndata['x']).to(device)
            cosine_sim = F.cosine_similarity(embedding1, embedding2).item()
            if cosine_sim > 0.99:
                isomorphic_same_count += 1  # Correctly classified as the same
            else:
                isomorphic_different_count += 1  # Incorrectly classified as different

            #final_embedding1 = calculate_integer_embedding(embedding1)
            #final_embedding2 = calculate_integer_embedding(embedding2)

            #if final_embedding1 == final_embedding2:
            #   isomorphic_same_count += 1
            #else:
            #    isomorphic_different_count += 1

    print(f"Correctly classified non-isomorphic pairs: {non_isomorphic_different_count}")
    print(f"Correctly classified isomorphic pairs: {isomorphic_same_count}")
    print(f"Incorrectly classified isomorphic pairs: {isomorphic_different_count}")

    return non_isomorphic_different_count, isomorphic_same_count, isomorphic_different_count

In [120]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GINConv, SumPooling, PNAConv
import networkx as nx
import random
import pickle
import numpy as np
import glob

# Define a GIN model
class GIN(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers):
        super(GIN, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Input layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(in_feats, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GINConv(
                nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim)
                ), 'sum'))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Output layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(hidden_dim, out_dim),
                nn.ReLU(),
                nn.Linear(out_dim, out_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(out_dim))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 100) / 100
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            h = layer(g, h)
            h = batch_norm(h)
            h = F.relu(h)
        g_embedding = self.pool(g, h)
        return g_embedding

# Define a PNA model
class PNA(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers, aggregators, scalers, deg):
        super(PNA, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(PNAConv(in_feats, hidden_dim, aggregators, scalers, deg))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(PNAConv(hidden_dim, hidden_dim, aggregators, scalers, deg))

        # Output layer
        self.layers.append(PNAConv(hidden_dim, out_dim, aggregators, scalers, deg))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 100) / 100
        for layer in self.layers:
            h = layer(g, h)
            h = F.relu(h)
        g_embedding = self.pool(g, h)
        return g_embedding



for filepath in glob.glob("../data/SR25/*SR25_dataset_dummy.pkl"):
    print(f"------------- Processing {filepath}...")

    # Load graphs to determine input_dim
    graphs = load_graphs_from_file(filepath)

    non_isomorphic_pairs, isomorphic_pairs = organize_pairs_sr25(graphs, 15, SR25_indices)

    if 'x' in graphs[0].ndata:
        input_dim = graphs[0].ndata['x'].size(1)  # Determine the input dimension dynamically
    else:
        # If 'x' does not exist, assume a default input dimension
        input_dim = 1


    # Initialize the models
    gin_model = GIN(input_dim, hidden_dim=32, out_dim=8, num_layers=4).to(device)
    pna_model = PNA(input_dim, hidden_dim=32, out_dim=8, num_layers=4,
                    aggregators=['mean', 'max', 'sum', 'min', 'std'],
                    scalers=['identity', 'amplification', 'attenuation'],
                    deg=torch.tensor([1.0]).to(device)).to(device)  # Example degree tensor

    # Compute the number of unique embeddings using PNA
    print("Using PNA model...")
    non_isomorphic_different_count, isomorphic_same_count, isomorphic_different_count = evaluate_model_with_pairs(non_isomorphic_pairs, isomorphic_pairs, pna_model, input_dim)

    # Compute the number of unique embeddings using GIN
    print("Using GIN model...")
    non_isomorphic_different_count, isomorphic_same_count, isomorphic_different_count = evaluate_model_with_pairs(non_isomorphic_pairs, isomorphic_pairs, gin_model, input_dim)



------------- Processing ../data/SR25/DE_SR25_dataset_dummy.pkl...
Using PNA model...
Epoch 1/100, Loss: 2.0000
Epoch 2/100, Loss: 2.0000
Epoch 3/100, Loss: 2.0000
Epoch 4/100, Loss: 2.0000
Epoch 5/100, Loss: 2.0000
Epoch 6/100, Loss: 2.0000
Epoch 7/100, Loss: 2.0000
Epoch 8/100, Loss: 2.0000
Epoch 9/100, Loss: 2.0000
Epoch 10/100, Loss: 2.0000
Epoch 11/100, Loss: 2.0000
Epoch 12/100, Loss: 2.0000
Epoch 13/100, Loss: 2.0000
Epoch 14/100, Loss: 2.0000
Epoch 15/100, Loss: 2.0000
Epoch 16/100, Loss: 2.0000
Epoch 17/100, Loss: 2.0000
Epoch 18/100, Loss: 2.0000
Epoch 19/100, Loss: 2.0000
Epoch 20/100, Loss: 2.0000
Epoch 21/100, Loss: 2.0000
Epoch 22/100, Loss: 2.0000
Epoch 23/100, Loss: 2.0000
Epoch 24/100, Loss: 2.0000
Epoch 25/100, Loss: 2.0000
Epoch 26/100, Loss: 2.0000
Epoch 27/100, Loss: 2.0000
Epoch 28/100, Loss: 2.0000
Epoch 29/100, Loss: 2.0000
Epoch 30/100, Loss: 2.0000
Epoch 31/100, Loss: 2.0000
Epoch 32/100, Loss: 2.0000
Epoch 33/100, Loss: 2.0000
Epoch 34/100, Loss: 2.0000
Epoch

In [119]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GINConv, SumPooling, PNAConv
import networkx as nx
import random
import pickle
import numpy as np
import glob

# Define a GIN model
class GIN(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers):
        super(GIN, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Input layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(in_feats, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GINConv(
                nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim)
                ), 'sum'))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Output layer
        self.layers.append(GINConv(
            nn.Sequential(
                nn.Linear(hidden_dim, out_dim),
                nn.ReLU(),
                nn.Linear(out_dim, out_dim)
            ), 'sum'))
        self.batch_norms.append(nn.BatchNorm1d(out_dim))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 1000) / 1000
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            h = layer(g, h)
            h = batch_norm(h)
            h = F.relu(h)
            #h = torch.round(h * 1000) / 1000
            h = h/torch.sum(h) * 10
        #g_embedding = self.pool(g, h)
        return h

# Define a PNA model
class PNA(nn.Module):
    def __init__(self, in_feats, hidden_dim, out_dim, num_layers, aggregators, scalers, deg):
        super(PNA, self).__init__()
        self.in_feats = in_feats
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(PNAConv(in_feats, hidden_dim, aggregators, scalers, deg))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(PNAConv(hidden_dim, hidden_dim, aggregators, scalers, deg))

        # Output layer
        self.layers.append(PNAConv(hidden_dim, out_dim, aggregators, scalers, deg))

        # Pooling layer
        self.pool = SumPooling()

    def forward(self, g, h):
        h = torch.round(h * 1000) / 1000
        for layer in self.layers:
            h = layer(g, h)
            h = F.relu(h)
            #h = torch.round(h * 1000) / 1000
            h = h/torch.sum(h) * 10
        #g_embedding = self.pool(g, h)
        return h

# Define a DeepSet model
class DeepSet(nn.Module):
    def __init__(self, input_dim):
        super(DeepSet, self).__init__()
        self.input_dim = input_dim  # Store input dimension
        # First neural network to map node features to node embeddings
        self.phi = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )
        # Second neural network to map summed node embeddings to final graph embedding
        self.rho = nn.Sequential(
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 8)  # Output a single value
        )

    def forward(self, g, h):
        # Apply the first neural network to each node feature
        h = self.phi(h)
        h = torch.round(h * 10000) / 10000  # Round to reduce precision
        # Sum the node embeddings to create a graph embedding
        g_embedding = h.sum(dim=0, keepdim=True)
        # Apply the second neural network to the summed embedding
        output = self.rho(g_embedding)*100
        return output


def calculate_integer_embedding(embedding):
    # Convert embedding to a flattened list of integers
    embedding = embedding.numpy()
    embedding = np.nan_to_num(embedding, nan=0.0) * 10
    embedding = embedding.astype(int)
    if embedding.ndim == 1:
        embedding = embedding.reshape(1, -1)  # Convert 1D array to a 2D array with one row

    column_sum = embedding.sum(axis=0)
    column_mean = embedding.mean(axis=0)
    column_min = embedding.min(axis=0)

    # Concatenate the results into a single 1D array
    final_embedding = np.concatenate((column_sum, column_mean, column_min))

    return int(np.sum((final_embedding.flatten()))*10)



# Save graphs to a file
def save_graphs_to_file(graphs, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(graphs, f)

# Load graphs from a file
def load_graphs_from_file(filepath):
    with open(filepath, 'rb') as f:
        graphs = pickle.load(f)
    return graphs

# Compute equivalence classes
def compute_equivalence_classes(filepath, model, input_dim):
    graphs = load_graphs_from_file(filepath)

    # Train the model for a single epoch with a random target
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    model.train()

    for epoch_in in range(10):
        for g in graphs:
            if 'x' not in g.ndata:
                g.ndata['x'] = torch.ones(g.number_of_nodes(), input_dim) 
            optimizer.zero_grad()

            output = model(g, g.ndata['x'])  # The output is a tensor of shape (1, out_dim)
            target = torch.randn_like(output)*100  # Target is a random tensor of the same shape as output

            loss = F.mse_loss(output, target.float(), reduction='mean')  # Ensure the target is a float tensor
            loss.backward()
            optimizer.step()

    embeddings = set()

    model.eval()
    with torch.no_grad():
        for g in graphs:
            if 'x' not in g.ndata:
                g.ndata['x'] = torch.ones(g.number_of_nodes(), input_dim)  # Initialize with ones if 'h' is missing
            embedding = model(g, g.ndata['x'])
            #print("embedding: ", embedding)
            final_embedding = calculate_integer_embedding(embedding.flatten())
            embeddings.add(final_embedding)

    print("embeddings: ", embeddings)
    return len(embeddings)

# Process all .pkl files in the current directory
for filepath in glob.glob("../data/SR25/*SR25_dataset*.pkl"):
    print(f"------------- Processing {filepath}...")

    # Load graphs to determine input_dim
    graphs = load_graphs_from_file(filepath)
    if 'x' in graphs[0].ndata:
        input_dim = graphs[0].ndata['x'].size(1)  # Determine the input dimension dynamically
    else:
        # If 'h' does not exist, assume a default input dimension
        input_dim = 1

    # Initialize the models
    gin_model = GIN(input_dim, hidden_dim=32, out_dim=8, num_layers=3)
    pna_model = PNA(input_dim, hidden_dim=32, out_dim=8, num_layers=3,
                    aggregators=['mean', 'max', 'sum', 'min', 'std'],
                    scalers=['identity', 'amplification', 'attenuation'],
                    deg=torch.tensor([1.0]))  # Example degree tensor

    # Compute the number of unique embeddings using GIN
    print("Computing equivalence classes using GIN model...")
    num_unique_embeddings_gin = compute_equivalence_classes(filepath, gin_model, input_dim)
    print(f"Number of unique embeddings with GIN: {num_unique_embeddings_gin}\n")

    # Compute the number of unique embeddings using PNA
    print("Computing equivalence classes using PNA model...")
    num_unique_embeddings_pna = compute_equivalence_classes(filepath, pna_model, input_dim)
    print(f"Number of unique embeddings with PNA: {num_unique_embeddings_pna}\n")


------------- Processing ../data/SR25/DE_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {1500}
Number of unique embeddings with GIN: 1

Computing equivalence classes using PNA model...
embeddings:  {2250}
Number of unique embeddings with PNA: 1

------------- Processing ../data/SR25/exN_SR25_dataset.pkl...
Computing equivalence classes using GIN model...
embeddings:  {120, 90, 60}
Number of unique embeddings with GIN: 3

Computing equivalence classes using PNA model...
embeddings:  {60, 30}
Number of unique embeddings with PNA: 2

------------- Processing ../data/SR25/exN_SR25_dataset_dummy.pkl...
Computing equivalence classes using GIN model...
embeddings:  {360, 330, 300, 390}
Number of unique embeddings with GIN: 4

Computing equivalence classes using PNA model...
embeddings:  {240, 300, 270}
Number of unique embeddings with PNA: 3

------------- Processing ../data/SR25/GLE_SR25_dataset.pkl...
Computing equivalence classes using GIN model...
